# Visualization contest test fixture creation

In [89]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import json

In [90]:
bd.projects.set_current("to_dataframe USEEIO")

In [91]:
datadir = Path("data")
datadir.mkdir(exist_ok=True)

Install US EEIO database

In [92]:
bi.useeio11()

USEEIO-1.1 already present


Write all processes and products to a dataframe

In [93]:
df = bd.Database("USEEIO-1.1").nodes_to_dataframe()

In [94]:
def safe_joiner(x):
    try:
        if not x or np.isnan(x):
            return None
    except TypeError:
        pass
    return "::".join(x)

In [95]:
df['categories'] = df['categories'].apply(safe_joiner)

In [96]:
def safe_converter(x):
    try:
        return str(int(x))
    except:
        return None

In [97]:
df['CAS number'] = df['CAS number'].apply(safe_converter)

In [98]:
df.to_csv(datadir / "useeio_nodes.csv", index=False)

Write all database edges to a dataframe

In [99]:
bd.Database("USEEIO-1.1").edges_to_dataframe().to_csv(datadir / "useeio_edges.csv", index=False)

Getting activity data


100%|██████████| 2649/2649 [00:00<00:00, 162463.43it/s]


Adding exchange data to activities


100%|██████████| 162926/162926 [00:06<00:00, 23739.12it/s]


Filling out exchange data


100%|██████████| 2649/2649 [00:00<00:00, 14629.87it/s]


Creating DataFrame
Compressing DataFrame


In [100]:
product = next(node for node in bd.Database("USEEIO-1.1") if node['type'] == 'product')

Generate an LCA object instance for use later

In [101]:
lca = bc.LCA({product: 1}, ('Impact Potential', 'ACID'))
lca.lci(factorize=True)
lca.lcia()

Write LCA matrices to dataframes

In [102]:
lca.to_dataframe("technosphere_matrix", cutoff=None, annotate=False).to_csv(datadir/ "useeio_technsophere_matrix.csv", index=False)

In [103]:
lca.to_dataframe("biosphere_matrix", cutoff=None, annotate=False).to_csv(datadir / "useeio_biosphere_matrix.csv", index=False)

Make dictionary of characterization matrices

In [104]:
c_matrices = {}

lciadir = (datadir / "characterization")
lciadir.mkdir(exist_ok=True)

for method in bd.methods:
    lca.switch_method(method)
    c_matrices[method] = lca.characterization_matrix.copy()
    lca.to_dataframe(
        "characterization_matrix", 
        cutoff=None, 
        annotate=False
    ).to_csv(
        lciadir / "useeio_characterization_matrix_{}.csv".format("_".join(method)), 
        index=False
    )

In [105]:
proddir = (datadir / "products")
proddir.mkdir(exist_ok=True)

Write characterized inventory LCA result matrices for all products and all impact categories in the US EEIO

In [106]:
lca_results = []

products = [node for node in bd.Database("USEEIO-1.1") if node['type'] == 'product']

for product in tqdm(products):
    row = {'name': product['name'], 'id': product.id}
    thisdir = (proddir / str(product.id))
    thisdir.mkdir(exist_ok=True)
    lca.lci({product.id: 1})

    for label, matrix in c_matrices.items():
        lca.characterization_matrix = matrix
        lca.lcia_calculation()
        
        row["_".join(label)] = lca.score
        lca.to_dataframe(
            "characterized_inventory", 
            cutoff=1000, 
            annotate=False
        ).to_csv(
            thisdir / "characterized_inventory_{}_{}.csv".format(product.id, "_".join(label)), 
            index=False,
        )
    
    lca_results.append(row)

100%|██████████| 388/388 [00:48<00:00,  8.04it/s]


Write dataframe with just the summed total scores for all products and all impact categories

In [107]:
pd.DataFrame(lca_results).to_csv(datadir/ "useeio_product_scores.csv", index=False)

Write the graph structure tracing the flow of impacts for four impact categories and the top five products in each impact category

In [108]:
graphdir = (datadir / "graphs")
graphdir.mkdir(exist_ok=True)

In [109]:
method_selection = [
    ('Impact Potential', 'GCC'),
    ('Impact Potential', 'HTOX'),
    ('Resource Use', 'LAND'),
    ('Resource Use', 'WATR'),
]

In [110]:
lca_results.sort(key=lambda x: x['Impact Potential_GCC'], reverse=True)

In [111]:
for selected_method in method_selection:
    print(selected_method)
    lca.characterization_matrix = c_matrices[selected_method]
    lca_results.sort(key=lambda x: x["_".join(selected_method)], reverse=True)    
    for row, _ in zip(lca_results, range(5)):
        lca.lcia({row['id']: 1})
        gt = bc.MultifunctionalGraphTraversal().calculate(lca, cutoff=0.0025)
        with open(graphdir / "{}-{}.json".format(row['id'], "_".join(selected_method)), "w") as f:
            json.dump(gt, f, sort_keys=True, indent=2)

('Impact Potential', 'GCC')
('Impact Potential', 'HTOX')
('Resource Use', 'LAND')
('Resource Use', 'WATR')
[{'name': 'Tobacco, cotton, sugarcane, peanuts, sugar beets, herbs and spices, and other crops; at farm', 'id': 408, 'Impact Potential_HRSP': 0.01679010962356698, 'Impact Potential_OZON': 1.4613218785979362e-07, 'Impact Potential_HNC': 6.300088182469611e-09, 'Resource Use_WATR': 3.66634732505426, 'Releases_METL': 2.8652950961340017e-07, 'Impact Potential_EUTR': 0.006872651655910232, 'Impact Potential_HTOX': 7.623387725150173e-09, 'Resource Use_LAND': 16.940097786517526, 'Resource Use_NREN': 10.638538641107576, 'Impact Potential_ETOX': 24.69978114959462, 'Releases_PEST': 0.00034368541118380785, 'Resource Use_REN': 0.36982391881300936, 'Resource Use_MINE': 0.47478104946291727, 'Impact Potential_GCC': 1.4950030480060912, 'Impact Potential_ACID': 0.0076254859381141024, 'Releases_HAPS': 0.0002854436392254584, 'Impact Potential_HC': 1.3232995731890368e-09, 'Impact Potential_SMOG': 0.069